# Median-based Bandits with Unbounded Rewards

We consider the class of upper confidence bound bandit algorithms for sequential experiment design problems.
We propose a policy, $\textit{Median-based Upper Confidence Bound}$ (M-UCB), based on the empirical median, that is robust to skewed distributions and outliers. In each round ${t}$ ($t > K$), pick an arm with index

$$argmax_{i \in \mathcal{K}} \underbrace{\hat{m}_{i, T_i(t-1)}}_{\substack{\text{Empirical} \\ \text{Median}}} + \beta  \underbrace{\left(\sqrt{2v_{i,t} \varepsilon_t} + 2 \varepsilon_t \sqrt{\frac{v_{i,t}}{T_i(t-1)}}\right)}_{\text{Confidence Width}},$$


where $T_i(t-1)$ is the number of times arm $i$ has been played during first $t-1$ rounds, exploration factor $\varepsilon_t = \alpha \log t$ with $\alpha$ controlling the exploration rate, hazard factor $v_{i,t} = \frac{4 }{T_i(t-1) \hat{L}_{i,T_i(t-1)}^2}$, with $\hat{L}_{i, T_i(t-1)}$ as the lower bound estimation of hazard rate for reward distribution of arm $i$ at the round $t$.
    $\beta$ is a hyper-parameter balancing the empirical median and confidence width.

In [ ]:
# direct to proper path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
import pickle
from prettytable import PrettyTable

from codes.UCB_discrete import M_UCB, UCB1, UCB_V, MV_LCB, Exp3, U_UCB, Median_of_Means_UCB, epsilon_greedy
from codes.Environment import Clinical_env
from codes.SimulatedGames import simulate
from codes.plots import plot_eva_for_clinical

%matplotlib inline

## Effect of Treatment on Cancer Survival

The clinical experiment is a classical example of bandit problems. The type of treatments needs to be determined when patients arrive sequentially and the effectiveness of treatments are initially unknown.
Survival analysis with medians and hazard rate are commonly used in clinical experiment analysis.
Decision-makers in clinical treatments need to manage risk and maximise the optimal choices (i.e. minimise the number of sub-optimal choices).
Furthermore, empirical medians (the basis of our approach) are robust against positively skewed distributions.

We use two clinical datasets from [Broad Institute Firehouse](https://gdac.broadinstitute.org/), namely the Glinoma dataset, and the Stomach and Esophageal carcinoma (STES) dataset.
These were chosen because they have a large number of samples.
For both datasets, there are two choices $\textit{Radiation}$ and $\textit{No Radiation}$,
and the reward is the number of days a patient survives (Survival Days).
As patients arrive sequentially, our goal is to minimise the sub-optimal treatment choices.

In [ ]:
FIXED_L = None
est_flag = True # estimate lower bound of hazard rate L

clinical_envs = defaultdict(list)
medians = defaultdict(list)
means = defaultdict(list)

num_rounds = 10000
num_exper = 50
num_arms = 2

In [ ]:
def read_data(data_path):
    """Read data from txt file.
    
    return
    -------------------------------------
    env_data: dataframe
         two columns: arms (yes; no); 
                     rewards (positive int, survival days)
    """
    data = pd.read_csv(data_path, delimiter= "\t", header = 0)
    data = data.T
    data.columns = data.iloc[0]
    data = data.drop(data.index[0])

    env_data = data[['radiation_therapy', 'days_to_death']]
    env_data = env_data.rename(columns = {'radiation_therapy': 'arms', 'days_to_death': 'rewards'})

    return env_data

In [ ]:
def setup_env(data, data_name):
    """Setup environment for clinical data.
    
    Parameters
    ----------------------------------------------------
    data: dataframe
        two columns: arms (yes; no); 
                     rewards (positive int, survival days)
    data_name: string
        name for clinical data, use for keys of dictionaries
        
    Return
    ----------------------------------------------------
    Rad: array 
        rewards (days of survival) when patients choose radiations
    NoRad: array
        rewards (days of survival) when patients choose no radiations
    """
    Rad = data.groupby(['arms']).get_group('yes')['rewards'] # Radiation 
    NoRad = data.groupby(['arms']).get_group('no')['rewards'] # No Radiation
    
    # Drop NaN
    Rad = np.asarray(Rad.dropna(), dtype = int) /30.0 # convert days to months
    NoRad = np.asarray(NoRad.dropna(), dtype = int) /30.0
    
    # Setup global variables
    clinical_envs[data_name] = [Clinical_env(NoRad), Clinical_env(Rad)]
    medians[data_name] = [np.median(NoRad),np.median(Rad)]
    
    return Rad, NoRad

In [ ]:
def show_data(Rad, NoRad, data_name):
    plt.figure(figsize=(5 * 3 , 4))
    plt.subplot(1,3,1)
    print('For ', data_name, ' dataset:')
    print('The number of samples with radiation: ', len(Rad))
    print('Median: ', np.median(Rad), ' Mean: ', np.mean(Rad))
    print()
    print('The number of samples without radiation: ', len(NoRad))
    print('Medians: ', np.median(NoRad), ' Mean: ', np.mean(NoRad))
    
    num_bins = int(max(max(Rad), max(NoRad))/6) # one bin represents half year
    #num_bins = 15
    sns.distplot(Rad, bins = num_bins, label = 'Radiation', norm_hist=True, kde = False)
    sns.distplot(NoRad, bins = num_bins, label = 'No Radiation', norm_hist=True, kde = False)
    
    plt.xlim(-5,110)
    plt.ylim(0, 15 * 1e-2)
    
    plt.xlabel('Survival Months')
    plt.ylabel('Frequency')
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    title_name = data_name + ' Treatment Histogram'
    plt.title(title_name)
    plt.legend()
    # plt.savefig(title_name + '.pdf', bbox_inches='tight')

### Glinoma Dataset

In [ ]:
Glin_path = '../Data/GBMLGG.clin.merged.picked.txt'
Glin_data = read_data(Glin_path)
Glin_data.head()

In [ ]:
Glin_Rad, Glin_NoRad = setup_env(Glin_data, 'Glioma')
show_data(Glin_Rad, Glin_NoRad, 'Glioma')

The reward distributions are positively skewed and have outliers in the right tail.
A small number of patients survived longer than 3000 days which are not shown in the histogram.

### STES Dataset

In [ ]:
STES_path = '../Data/STES.clin.merged.picked.txt'
STES_data = read_data(STES_path)
STES_data.head()

In [ ]:
STES_Rad, STES_NoRad = setup_env(STES_data, 'STES')
show_data(STES_Rad, STES_NoRad, 'STES')

### Experiments

We repeat our experiment 50 times, each with 10,000 rounds where we sample from
the database of records with replacement.

In [ ]:
def run_exper(rewards_env, hyperpara_list, num_exper, num_rounds, policy, summary_stats, est_flags = [est_flag]):
    for key in rewards_env.keys():
        for hyperpara in hyperpara_list:
            for est_flag in est_flags:
                name = key + '_' + str(num_exper) + '_' + str(num_rounds)
                policy_name = str(policy).split('.')[-1].split('\'')[0] + '-'
                subname = policy_name + str(hyperpara)
                print(name + subname)
                p = IntProgress(max = num_exper)
                p.description = 'Running'
                display(p)
                results[name][subname] = simulate(rewards_env[key], summary_stats[key], policy, num_exper, num_rounds, est_flag, hyperpara, FIXED_L, p)
                
def run_exper_diff_para(rewards_env, hyperpara_list, num_exper, num_rounds, policy, summary_stats, est_flags = [est_flag]):
    for key in rewards_env.keys():
        for est_flag in est_flags:
            name = key + '_' + str(num_exper) + '_' + str(num_rounds)
            
            # setting different parameters for different datasets
            if key == 'Glioma':
                hyperpara = hyperpara_list[0]
            elif key == 'STES':
                hyperpara = hyperpara_list[-1]
            policy_name = str(policy).split('.')[-1].split('\'')[0] + '-'

            subname = policy_name + str(hyperpara)
            print(name + subname)
            p = IntProgress(max = num_exper)
            p.description = 'Running'
            display(p)
            results[name][subname]= simulate(rewards_env[key], summary_stats[key], policy, num_exper, num_rounds, est_flag, hyperpara, FIXED_L, p)

In [ ]:
results = defaultdict(dict)

In [ ]:
# Our Algorithm: M-UCB

hyperpara_list = [[2, 1, 10], [2, 1, 10]]
run_exper_diff_para(clinical_envs, hyperpara_list, num_exper, num_rounds, M_UCB, medians)

#--------------------------------------------------------------------------------------------------------

# Benchmark Algorithms

# U-UCB

hyperpara_list = [[4]]
run_exper(clinical_envs, hyperpara_list, num_exper, num_rounds, U_UCB, medians)

# epsilon-greedy

hyperpara_list = [[0.1]]
run_exper(clinical_envs, hyperpara_list, num_exper, num_rounds, epsilon_greedy, medians)

# Median_of_means_UCB

hyperpara_list = [[10, 1, 1]]
run_exper(clinical_envs, hyperpara_list, num_exper, num_rounds, Median_of_Means_UCB, medians) 
# median and mean gives the same order

# Exp3

hyperpara_list = [[0.5, 0, 180], [0.5, 0, 100]]
run_exper_diff_para(clinical_envs, hyperpara_list, num_exper, num_rounds, Exp3, medians)
# median and mean gives the same order

In [ ]:
import pickle

with open('clinical.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
plot_eva_for_clinical(results, 'sd')

A small number of patients survived longer than 100 months are not shown in the histogram. Each bin represents half a year of survival. Note the median and mean give same order for both datasets. 
Our policy outperforms other algorithms, where U-UCB has a similar but slightly worse performance. The reason the difference between our method M-UCB and U-UCB is smaller here is that the gap between medians of different arms is bigger here.
We choose a smaller parameter $\alpha$, since less exploration is needed when the gap of medians is big. 